In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/Community_Review_Generated_Dish_Images.csv")
print(df.shape)
df.head()

(5948, 20)


,Country,Rater_id,Dish name,Model,Generation_count,Would you associate the dish_name with your home and/or culture?,How familiar are you with the dish_name?,Is this an image of food?,Is this an image of the dish_name,Please rate quality of the image,"In your opinion, how closely does this image match what you’d expect an image of the dish_name to be?",Please provide an explanation for your answer ( especially important if the image doesn't resemble the dish in question),What do you see in the image? ( if this is not an image of food you can submit the form after answering this question),Please check any of the below that apply; presentation of the dish,Please check any of the below that apply; utensils,Please check any of the below that apply; background,Please check any of the below that apply; others,Familiarity_level,Image_match,Good_quality
0,United States,6,agua fresca,Stable Diffusion,0.0,Yes,"Somewhat familiar, I looked it up or asked som...",Yes,No,1.0,"The image resembles the dish_name, but it is n...",This is a collage of shapes and colors and dri...,NaN,NaN,The utensils are correct,The dish is shown served on a weird background,NaN,Somewhat familiar,Resembles but defies expectation,No
1,United States,6,agua fresca,Dalle3,0.0,Yes,"Somewhat familiar, I looked it up or asked som...",Yes,Yes,5.0,"The image resembles the dish_name, but it is n...",This is a cartoon/line drawing of many differe...,NaN,The dish is shown served in a way that I’m use...,NaN,NaN,NaN,Somewhat familiar,Resembles but defies expectation,Yes
2,United States,6,agua fresca,Dalle2,0.0,Yes,"Somewhat familiar, I looked it up or asked som...",Yes,No,2.0,"The image resembles the dish_name, but it is n...",These are drinks but agua fresca isn't thick/c...,NaN,NaN,The utensils are correct,The dish is shown served on a weird background,NaN,Somewhat familiar,Resembles but defies expectation,No
3,United States,5,agua fresca,Dalle2,0.0,No,"Somewhat familiar, I looked it up or asked som...",Yes,Yes,4.0,"The image resembles the dish_name, but it is n...",The picture is of a drink that resembles agua ...,NaN,The image sort of looks like the {dish_name} b...,NaN,NaN,NaN,Somewhat familiar,Resembles but defies expectation,Yes
4,United States,5,agua fresca,Dalle3,0.0,No,"Very familiar, I eat this dish regularly",Yes,No,3.0,"The image resembles the dish_name, but it is n...",This is an illustration of a clear beverage wi...,NaN,The image sort of looks like the {dish_name} b...,The utensils are correct,NaN,NaN,Very familiar,Resembles but defies expectation,Yes


In [3]:
columns_rename = {
    "Dish Name": "dish_name",
    "Would you associate the dish_name with your home and/or culture?": "associate_with_home",
    "How familiar are you with the dish_name?": "raw_familiarity_level",
    "Please rate quality of the image": "image_quality_rating",
    "What do you see in the image? ( if this is not an image of food you can submit the form after answering this question)": "image_description",
    "Is this an image of food?": "is_food",
    "Is this an image of the dish_name": "is_exact_dish",
    "In your opinion, how closely does this image match what you’d expect an image of the dish_name to be?": "raw_image_match",
    "Please provide an explanation for your answer ( especially important if the image doesn't resemble the dish in question)": "image_match_explanation",
    "Please check any of the below that apply; Presentation of the dish": "presentation_of_image",
    "Please check any of the below that apply; Utensils": "utensil_of_image",
    "Please check any of the below that apply; Background": "background_of_image",
    "Please check any of the below that apply; Others": "others_of_image",
    "If any of the checkboxes have been checked:\nPlease elaborate where needed": "checkbox_explanation"
  }

In [4]:
df = df.rename(columns=columns_rename)
df.columns = df.columns.str.lower()
df.columns

Index(['country', 'rater_id', 'dish name', 'model', 'generation_count',
       'associate_with_home', 'raw_familiarity_level', 'is_food',
       'is_exact_dish', 'image_quality_rating', 'raw_image_match',
       'image_match_explanation', 'image_description',
       'please check any of the below that apply; presentation of the dish',
       'please check any of the below that apply; utensils',
       'please check any of the below that apply; background',
       'please check any of the below that apply; others', 'familiarity_level',
       'image_match', 'good_quality'],
      dtype='object')

In [5]:
out_list=[]

# Calculate the proportion
for query, name in zip(['is_food', 'is_exact_dish', 'good_quality'], ['%Food generated', '%Correct food', "%Good quality"]):
  total_counts = df.groupby(['country', 'model']).size()
  query_counts = df[df[query] == 'Yes'].groupby(['country', 'model']).size()

  proportion= query_counts / total_counts



  # Creating a dataframe for the result
  result_subset = pd.DataFrame({
      'proportion': proportion
  }).reset_index()

  out = result_subset.pivot(columns='model', index='country', values='proportion')
  out = out.fillna(0)
  out_list.append(pd.concat([out], keys=[name], axis=1))


In [6]:
tablex = pd.concat(out_list, axis=1).round(3)
tablex

%Food generated                         %Correct food         \
model                  Dalle2 Dalle3 Stable Diffusion        Dalle2 Dalle3   
country                                                                      
Algeria                 0.767  0.931            0.528         0.038  0.201   
Cameroon                0.756  0.947            0.866         0.181  0.146   
Kenya                   0.883  0.814            0.700         0.228  0.393   
Nigeria                 0.822  0.953            0.668         0.180  0.156   
South Africa            0.930  0.811            0.591         0.315  0.545   
United States           1.000  1.000            0.940         0.770  0.820   

                               %Good quality                          
model         Stable Diffusion        Dalle2 Dalle3 Stable Diffusion  
country                                                               
Algeria                  0.024         0.874  1.000            0.410  
Cameroon                 0.030         0.975  0.996            0.991  
Kenya                    0.162         0.793  0.993            0.469  
Nigeria                  0.044         0.656  0.589            0.350  
South Africa             0.161         0.980  0.845            0.365  
United States            0.438         0.633  0.970            0.410

In [7]:
# print(tablex.to_latex())